In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [3]:
y_train = pd.read_csv('data\\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(71210, 13129)

In [4]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.CenterCrop((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
x_train = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val = r'D:\Machine Learning\deepfake-detection\data\validation_set'
x_test = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [6]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        self.fc = nn.Linear(2048, 1)

In [7]:
#model = models.resnext50_32x4d(pretrained=True)
#model.fc = nn.Linear(2048, 1)

In [8]:
model = MyResNeXt()
loss =  torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.) 
#optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3) 
#optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
%%script false

model.load_state_dict(torch.load("pretrained/resnext.pth", map_location=device))
#_ = model.eval()

Couldn't find program: 'false'


In [10]:
#%%script false

exceptions = ['layer4.0.conv1.weight']

for name, param in model.named_parameters():
    for e in exceptions:
        if e in name:
            param.requires_grad = False
        else:
            param.requires_grad = True

Couldn't find program: 'false'


In [11]:
model = model.to(device)

In [12]:
%%script false

'''
ResNext50 - 0.8975 mean | Test - 0.77
'''

validate_vid_bf(model, x_test, y_train, loss, accuracy_sigmoid_mean, device, frames_per_video, face_extractor, print_results=True)

Couldn't find program: 'false'


In [ ]:
metric, lost = train_vid_bf(model, loss, optimizer, scheduler, x_train, x_test, y_train, accuracy_sigmoid_mean, device, 
                            face_extractor, 
                            frames=2,
                            epochs=30, 
                            batch_size=10, 
                            del_net=False, 
                            useScheduler=True,
                            checkpoint=0.89,
                           )


Validation: metrics  0.4925
Train: metrics  0.7234346318580461 loss  0.5550456396788359


In [ ]:
from operator import itemgetter
min_lost = min(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = max(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
validate_vi1d(model, x_val, y_train, loss, accuracy_b, device, 30, fast_mtcnn, transforms, 
             show_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, print_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, show_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, show_graphic=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
#torch.save(model.state_dict(), '0.657.pth')